Below is the training of classifier for each choice of c value using the primal SVM with slack formation 

optimization problem: min (w, b, slack) 1/2||w||^2 + c sum(slack)
                      such that: 
                        - y_i(wx_i+b) >= 1 - slack_i, for all i
                        - slack_i >= 0, for all i

File Paths: 
- training data: spam_data/spam_train.data
- validation data: spam_data/spam_validation.data
- test data: spam_data/spam_test.data

In [1]:
import numpy as np
import cvxopt
import cvxopt.solvers

#read file and store data to X and Y 
def read_data(filename): 
    file = open(filename, 'r', encoding='utf-8-sig')
    dataset = []
    for line in file:
        data = line.split(',')
        x_data = [float(x) for x in data[:57]] 
        y_data = int(data[57]) 
        dataset.append((x_data, y_data))

    X = np.array([x for x, y in dataset])
    Y = np.array([y for x, y in dataset])
    Y = np.where(Y == 0, -1, Y) #replace 0 with -1

    return X, Y

#training model for the train_data and get an optimal w and b
def train(c): 
    train_X, train_Y = read_data("spam_data/spam_train.data")

    num_of_data, num_of_features = train_X.shape

    #P matrix (identity matrix of w's, 0 for b and slack)
    P_matrix = np.identity(num_of_features) #identity matrix
    P_matrix = np.pad(P_matrix, ((0, 2), (0, 0)), mode='constant', constant_values=0) #add 2 new rows of 0's to the end
    P_matrix = np.pad(P_matrix, ((0, 0), (0, 2)), mode='constant', constant_values=0) #add 2 new cols of 0's to the end 

    #q vector (0's for w's and b, c for slack)
    q_vector = np.zeros(num_of_features+1) #col of 0's
    q_vector = np.append(q_vector, c) #add c value 

    #G matrix 
    train_Y = train_Y.reshape(train_Y.shape[0],1) 
    G_matrix = -np.multiply(train_Y, train_X) #-yixi
    bcol = np.array([-y for y in train_Y]) #-yi
    G_matrix = np.hstack((G_matrix, bcol)) #add bcol to G matrix
    G_matrix = np.pad(G_matrix, ((0, num_of_data), (0, 0)), mode='constant', constant_values=0) #add 3000 new rows of 0's to the end (for second equality)
    scol = -np.ones(num_of_data*2).reshape(num_of_data*2, 1) #col of slack variable (-1)
    G_matrix = np.hstack((G_matrix, scol)) #add bcol to G matrix

    #h vector
    h_vector = -np.ones(num_of_data) #col of -1's
    new_col = np.zeros(num_of_data) #col of 0's
    h_vector = np.append(h_vector, new_col)

    P = cvxopt.matrix(P_matrix)
    q = cvxopt.matrix(q_vector)
    G = cvxopt.matrix(G_matrix)
    h = cvxopt.matrix(h_vector) 

    cvxopt.solvers.options['show_progress'] = False #disable progess to console 
    sol = cvxopt.solvers.qp(P, q, G, h) 
    sol_arr = np.array(sol['x'])

    w = sol_arr[:num_of_features]
    b = sol_arr[num_of_features]

    return w, b


#main(): 
X, Y = read_data("spam_data/spam_test.data")
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

#compute accuracy for each c value 
for c in C: 
    w, b = train(c)
    correct_predictions = 0
    total_data = X.shape[0]

    for x, y in zip(X, Y):
        result = y * (np.dot(w.reshape(1, w.shape[0]),x) + b)
        if (result > 0): #correct classification 
            correct_predictions += 1
    
    accuracy = (correct_predictions / total_data) * 100
    print("c:", c)
    print("accuracy:", accuracy, "%")

c: 0.001
accuracy: 86.89138576779027 %
c: 0.01
accuracy: 85.89263420724095 %
c: 0.1
accuracy: 84.76903870162296 %
c: 1
accuracy: 84.14481897627965 %
c: 10
accuracy: 85.0187265917603 %
c: 100
accuracy: 84.14481897627965 %
c: 1000
accuracy: 84.76903870162296 %
